In [1]:
import pandas as pd
import os

# Data Merging

In [2]:
fuel_data = pd.read_csv("../raw_data/fuel_data.csv")
fuel_data = fuel_data.drop(columns=["Unnamed: 0"])
inflation_data = pd.read_csv("../raw_data/inflation_data.csv")
inflation_data = inflation_data.drop(columns=["Unnamed: 0"])
myr_data = pd.read_csv("../raw_data/myr_to_usd.csv")
opr_data = pd.read_csv("../raw_data/overnight_policy_rate.csv")


# Convert to Date Format

#fuel data
fuel_data["date"] = pd.to_datetime(fuel_data["date"])

#inflation data
inflation_data["date"] = pd.to_datetime(inflation_data["date"])

#myr to usd data
myr_data["date"] = pd.to_datetime(myr_data["Date"], dayfirst=True)
myr_data = myr_data.drop(columns=["Date"])

#opr data
opr_data["date"] = pd.to_datetime(opr_data["Date"], dayfirst=True)
opr_data = opr_data.drop(columns=["Date"])



inflation_data = inflation_data[ inflation_data["division"] == "overall" ]
fuel_data = fuel_data[ fuel_data["series_type"] == "level" ]


In [3]:
# Merge all by "date" column
merged = fuel_data.merge(myr_data, on="date", how="outer") \
                  .merge(inflation_data, on="date", how="outer") \
                  .merge(opr_data, on="date", how="outer") \

merged.to_csv("../raw_data/merged_data.csv")
merged.head()


,date,ron95,ron97,diesel,ron95_skps,series_type,ron95_budi95,diesel_eastmsia,USD,index,state,division,Rate
0,2004-05-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7
1,2004-08-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7
2,2004-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7
3,2005-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7
4,2005-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7


# Data Cleaning

### 1. Date to datetime

In [4]:
raw_data = pd.read_csv("../raw_data/merged_data.csv")
raw_data = raw_data.drop(columns=["Unnamed: 0"])
raw_data["date"] = pd.to_datetime(raw_data["date"])
raw_data

,date,ron95,ron97,diesel,ron95_skps,series_type,ron95_budi95,diesel_eastmsia,USD,index,state,division,Rate
0,2004-05-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7
1,2004-08-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7
2,2004-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7
3,2005-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7
4,2005-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5913,2026-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9695,NaN,NaN,NaN,NaN
5914,2026-01-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9575,NaN,NaN,NaN,NaN
5915,2026-01-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9205,NaN,NaN,NaN,NaN
5916,2026-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9330,NaN,NaN,NaN,NaN


### 2. Fixing null values

Forward-fill operation

In [5]:
cols_to_fill = ["ron95", "ron97", "diesel", "ron95_budi95","Rate", "USD"]
raw_data[cols_to_fill] = raw_data[cols_to_fill].ffill()
raw_data.head()
raw_data.to_csv("../raw_data/raw_data.csv")

### 3. Filter raw_data to only include records dated after the earliest fuel_data entry.

In [6]:
oldest_date = fuel_data["date"].min()
print("Oldest date:", oldest_date)
raw_data = raw_data[raw_data["date"] >= oldest_date]
raw_data.sort_values("date")

Oldest date: 2017-03-30 00:00:00


,date,ron95,ron97,diesel,ron95_skps,series_type,ron95_budi95,diesel_eastmsia,USD,index,state,division,Rate
1998,2017-03-30,2.13,2.41,2.11,NaN,level,NaN,2.11,4.4225,NaN,NaN,NaN,3.00
1999,2017-03-31,2.13,2.41,2.11,NaN,NaN,NaN,NaN,4.4265,NaN,NaN,NaN,3.00
2015,2017-04-01,2.13,2.41,2.11,NaN,NaN,NaN,NaN,4.4265,118.6,W.P. Putrajaya,overall,3.00
2014,2017-04-01,2.13,2.41,2.11,NaN,NaN,NaN,NaN,4.4265,117.5,W.P. Labuan,overall,3.00
2013,2017-04-01,2.13,2.41,2.11,NaN,NaN,NaN,NaN,4.4265,119.5,W.P. Kuala Lumpur,overall,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5913,2026-01-26,2.54,3.10,2.88,NaN,NaN,1.99,NaN,3.9695,NaN,NaN,NaN,2.75
5914,2026-01-27,2.54,3.10,2.88,NaN,NaN,1.99,NaN,3.9575,NaN,NaN,NaN,2.75
5915,2026-01-28,2.54,3.10,2.88,NaN,NaN,1.99,NaN,3.9205,NaN,NaN,NaN,2.75
5916,2026-01-29,2.54,3.10,2.88,NaN,NaN,1.99,NaN,3.9330,NaN,NaN,NaN,2.75


### 4. Remove unused columns

In [7]:
raw_data = raw_data.drop(columns=["ron95_skps", "series_type","diesel_eastmsia"])

### 5. If ron95_budi95 is NaN, use ron95 value for the same date


In [8]:
raw_data["ron95_budi95"] = raw_data["ron95_budi95"].fillna(raw_data["ron95"])
raw_data.sort_values("date")

,date,ron95,ron97,diesel,ron95_budi95,USD,index,state,division,Rate
1998,2017-03-30,2.13,2.41,2.11,2.13,4.4225,NaN,NaN,NaN,3.00
1999,2017-03-31,2.13,2.41,2.11,2.13,4.4265,NaN,NaN,NaN,3.00
2015,2017-04-01,2.13,2.41,2.11,2.13,4.4265,118.6,W.P. Putrajaya,overall,3.00
2014,2017-04-01,2.13,2.41,2.11,2.13,4.4265,117.5,W.P. Labuan,overall,3.00
2013,2017-04-01,2.13,2.41,2.11,2.13,4.4265,119.5,W.P. Kuala Lumpur,overall,3.00
...,...,...,...,...,...,...,...,...,...,...
5913,2026-01-26,2.54,3.10,2.88,1.99,3.9695,NaN,NaN,NaN,2.75
5914,2026-01-27,2.54,3.10,2.88,1.99,3.9575,NaN,NaN,NaN,2.75
5915,2026-01-28,2.54,3.10,2.88,1.99,3.9205,NaN,NaN,NaN,2.75
5916,2026-01-29,2.54,3.10,2.88,1.99,3.9330,NaN,NaN,NaN,2.75


### 6. Removing null values

In [9]:
raw_data = raw_data.dropna(subset=["state"])
raw_data.head()

,date,ron95,ron97,diesel,ron95_budi95,USD,index,state,division,Rate
2000,2017-04-01,2.13,2.41,2.11,2.13,4.4265,122.800000,Johor,overall,3.0
2001,2017-04-01,2.13,2.41,2.11,2.13,4.4265,118.600000,Kedah,overall,3.0
2002,2017-04-01,2.13,2.41,2.11,2.13,4.4265,118.724922,Kelantan,overall,3.0
2003,2017-04-01,2.13,2.41,2.11,2.13,4.4265,119.100000,Melaka,overall,3.0
2004,2017-04-01,2.13,2.41,2.11,2.13,4.4265,120.000000,Negeri Sembilan,overall,3.0


In [10]:
raw_data.sort_values("date")

,date,ron95,ron97,diesel,ron95_budi95,USD,index,state,division,Rate
2000,2017-04-01,2.13,2.41,2.11,2.13,4.4265,122.8,Johor,overall,3.00
2015,2017-04-01,2.13,2.41,2.11,2.13,4.4265,118.6,W.P. Putrajaya,overall,3.00
2014,2017-04-01,2.13,2.41,2.11,2.13,4.4265,117.5,W.P. Labuan,overall,3.00
2013,2017-04-01,2.13,2.41,2.11,2.13,4.4265,119.5,W.P. Kuala Lumpur,overall,3.00
2012,2017-04-01,2.13,2.41,2.11,2.13,4.4265,115.9,Terengganu,overall,3.00
...,...,...,...,...,...,...,...,...,...,...
5859,2025-12-01,2.63,3.26,3.08,1.99,4.1340,128.8,Kedah,overall,2.75
5858,2025-12-01,2.63,3.26,3.08,1.99,4.1340,140.1,Johor,overall,2.75
5872,2025-12-01,2.63,3.26,3.08,1.99,4.1340,125.4,W.P. Labuan,overall,2.75
5864,2025-12-01,2.63,3.26,3.08,1.99,4.1340,129.4,Perak,overall,2.75


In [11]:
raw_data.to_csv("../data/cleaned_data.csv")